# Converting a LightGBM model to ONNX

In this notebook we create a resuable pipeline component that will convert LightGBM models to ONNX format for use in a production-inference environment.<br>

We'll be using a pickled model from the **[LightGBM demo](https://github.com/mlrun/demos/blob/master/lightgbm/README.md)**. We would also like to append this to an existing pipeline, either directly in the pipeline building stage, or preferrably, as an independent function/service that will trigger an lgbm model conversion.

    !pip uninstall -y mlrun
    !pip install -U -q mlrun
    !pip install -U -q pandas numpy
    !pip install -U -q lightgbm
    !pip install -U -q joblib cloudpickle dill
    !pip install -U -q scikit-learn
    !pip install -U -q nuclio-jupyter

    import os
    os.environ['ONNX_ML'] = '1'

    !pip install -U -q onnxruntime
    !pip install -U -q git+https://github.com/microsoft/onnxconverter-common
    !pip install -U -q git+https://github.com/onnx/onnxmltools
    !conda install -y pydot python-graphviz

In [2]:
import mlrun
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [3]:
import os
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier

from skl2onnx import (convert_sklearn, 
                      update_registered_converter)

from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes  # noqa
from onnxmltools.convert.lightgbm.operator_converters.LightGbm import convert_lightgbm  # noqa
from skl2onnx.common.data_types import FloatTensorType

In [4]:
update_registered_converter(LGBMClassifier, 'LightGbmLGBMClassifier',
                            calculate_linear_classifier_output_shapes,
                            convert_lightgbm)

In [5]:
# todo: test lightgb, with pickle/dill...
from joblib import load, dump

In [ ]:
# load lightgbm demo model

In [6]:
clf = load(open('/User/mlrun/lightgbm/lightgbm.model.pkl', 'rb'))

In [8]:
type(clf)

lightgbm.sklearn.LGBMClassifier

In [9]:
model = onnxmltools.convert_lightgbm(clf.booster_, 
                                     name='LightGBM', 
                                     initial_types=[('input', FloatTensorType([1, 28]))])
onnxmltools.utils.save_model(model, 'LightGBM.onnx')

The maximum opset needed by this model is only 1.
The maximum opset needed by this model is only 9.


In [ ]:
model_onnx = convert_sklearn(clf, 'lightgbm_model',
                             [('input', FloatTensorType([None, 28]))])

In [ ]:
# And save.
with open("lightgbm_model.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())